3. Multi-headed: Multiple Input Series를 개선시킨 모델. (개별로 처리)

In [11]:
from tensorflow.python.client import device_lib
import keras.backend.tensorflow_backend as K
#print(device_lib.list_local_devices())

from numpy import array
from numpy import hstack
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers.merge import concatenate

In [6]:
# split a multivariate sequence into samples
def split_sequences_multi(sequences, n_steps):
    X, y = list(), list()
    len_ = len(sequences)
    for i in range(len_):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1] #마지막열이 y이기떄문에 -1
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [7]:
#예제 데이터 셋 정의
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

In [8]:
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
# choose a number of time steps
n_steps = 3
# convert into input/output
X, y = split_sequences_multi(dataset, n_steps)

In [9]:
######## separate input data ########
X1 = X[:, :, 0]
X2 = X[:, :, 1]

In [12]:
# define model
with K.tf.device('/cpu:0'): #'/gpu:0'
    # first input model
    visible1 = Input(shape=(n_steps,))
    dense1 = Dense(100, activation='relu')(visible1)

    # second input model
    visible2 = Input(shape=(n_steps,))
    dense2 = Dense(100, activation='relu')(visible2)

    # merge input models
    merge = concatenate([dense1, dense2])
    output = Dense(1)(merge)
    model = Model(inputs=[visible1, visible2], outputs=output)
    model.compile(optimizer='adam', loss='mse')

    # fit model
    model.fit([X1, X2], y, epochs=2000, verbose=0)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [13]:
# 예측
x_input = array([[80, 85], [90, 95], [100, 105]])
x1 = x_input[:, 0].reshape((1, n_steps))
x2 = x_input[:, 1].reshape((1, n_steps))
yhat = model.predict([x1, x2], verbose=0)
print(yhat)

[[205.03804]]
